In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Tikhonov regularisation using CGLS and block framework

This exercise introduces Tikhonov regularisation and explains how this is implemented in the CIL framework using the so-called block framework.

In the previous exercise, it was seen how CGLS could be used to determine a reconstruction based on the least squares reconstruction problem. It was seen that in case of noisy data, the least squares solution obtained by running until convergence is not desirable due to a high amount of noise. The number of iterations was seen to have a regularising effect, with the smooth, low-frequency components of the image recovered in the first iterations, while high-frequency components of the image such as edges were recovered later. Unfortunately, noise also kicks in, and one needs to pick the number of iterations that best balances the sharpness and amount of noise. As such, the regularising effect is implicitly obtained by choosing the number of iterations to run and never actually running until converged to the least squares solution.

Tikhonov regularization is more explicit in that a regularization term is added to the least squares fitting term, specifically a squared 2-norm. This problem should now be solved to convergence instead of using the number of iterations as implicit regularising effect. Instead, a parameter, the regularization parameter, balances the emphasis on fitting the data and enforcing the regularity and must be chosen to provide the best trade-off.

Tikhonov regularization tends to offer reduction of noise in the reconstruction, at the price of some blurring. This will be seen in what follows.

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularization parameter and operator in regulariser.

First, all imports required are carried out. This included tools from the ccpi.framework and ccpi.optimisation modules, as well as test image generation tools in the tomophantom library and standard imports such as numpy.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ccpi.framework import ImageGeometry, ImageData 
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import BlockDataContainer
from ccpi.framework import TestData

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity, FiniteDiff

from ccpi.astra.operators import AstraProjectorSimple 

import tomophantom
from tomophantom import TomoP2D
import scipy
import numpy as np    

import matplotlib.pyplot as plt

from utilities import plotter2D

import os

### Setting up the dataset - 2D

A 2D parallel beam case will be set. At first, the parameters of acquisition geometry and the image geometry are specified:

In [ ]:
#set up acquisition geometry
number_pixels_x = 1024
number_projections = 180
angles = np.linspace(0, np.pi, number_projections, dtype=np.float32)
ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)

#set up image geometry
num_voxels_xy = 1024
ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)

Then a the classic Shepp-Logan test image is loaded in from tomophantom, along with an analytically computed sinogram data set of it. The pixel values are rescaled for the purpose of Poisson noise simulation in a subsequent step.

In [ ]:
# Load Shepp-Logan phantom 
model = 1
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

#tomophantom takes angular input in degrees
phantom_2D = TomoP2D.Model(model, num_voxels_xy, path_library2D)
phantom_sino = TomoP2D.ModelSino(model, num_voxels_xy, number_pixels_x, angles*180./np.pi, path_library2D)

#rescale the tomophantom data, set the max absoobtion to 25%
set_ratio_absorption = 0.25
new_max_value = -np.log(set_ratio_absorption)
sino_max = np.amax(phantom_sino)
scale = new_max_value/sino_max

#allocate the image data container and copy the dataset in
#this is only used as a reference to the ground truth
model = ig.allocate(0)
model.fill(phantom_2D*scale)

#allocate the acquisition data container and copy the sinogram in
sinogram = ag.allocate(0)
sinogram.fill(phantom_sino*scale)

plots = [model, sinogram]
titles = ["Ground truth", "sinogram"]
plotter2D(plots,titles,fix_range=False, stretch_y=True)

Next Poisson noise is added. The severity of the noise can be adjusted by changing the background_counts variable. The simulated clean and noisy sinograms are displayed side by side as images.

In [ ]:
background_counts = 500 #lower counts will increase the noise
counts = float(background_counts) * np.exp(-sinogram.as_array())
noisy_counts = np.random.poisson(counts)
sino_out = -np.log(noisy_counts/background_counts)

sinogram_noisy = ag.allocate()
sinogram_noisy.fill(sino_out)

In [ ]:
plots = [sinogram, sinogram_noisy]
titles = ["sinogram", "sinogram noisy"]
plotter2D(plots,titles,fix_range=False, stretch_y=True)

<a id="section_CGLS_simple"></a>
### Reconstruct using unregularised CGLS

Before describing Tikhonov regularization, we recall the problem solved by CGLS:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}A u - b\end{Vmatrix}^2_2$$

where,

- $A$ is the projection operator

- $b$ is the acquired data

- $u$ is the unknown image to be determined

In the solution provided by CGLS the low frequency components tend to converge faster than the high frequency components. This means we need to control the number of iterations carefully to select the optimimum solution.

To solve using CGLS (and Tikhonov afterwards) we define the operator A and can choose between a CPU or GPU back-end:

In [ ]:
device = "gpu"
operator = AstraProjectorSimple(ig, ag, device)

We define the data b to be used in CGLS, and can here choose between the clean or the noisy sinogram data:

In [ ]:
data = sinogram
#data = sinogram_noisy

Set up the CGLS algorithm, including specifying its initial point to start from:

In [ ]:
x_init = ig.allocate(0)
cgls_simple = CGLS(x_init=x_init, operator=operator, data=data, update_objective_interval = 10)
cgls_simple.max_iteration = 1000

Once set up, we can run the algorithm for a specified number of iterations:

In [ ]:
cgls_simple.run(20, verbose = True)
#cgls_simple.run(1, verbose = True)

Display the resulting image from CGLS, along with its difference image with the original ground truth image:

In [ ]:
CGLS_simple_output = cgls_simple.get_output()

plots = [CGLS_simple_output, CGLS_simple_output - model]
titles = ["CGLS reconstruction","Difference from ground truth" ]
plotter2D(plots,titles,fix_range=True)

<span style="color:red;font-size:larger">**Exercise 1:**</span> Repeat this with the noisy dataset. Remember you can change the number of iterations to run between outputs. Try to stop the algorithm before the solution starts to diverge. [go to section start](#section_CGLS_simple)

### Tikhonov regularization using CGLS

#### Regularisation

Noisy datasets lead to an ill-posed problem. If we try to solve these using CGLS we end up with an unstable solution. Regularisation adds information in order for us to solve the problem.

#### Tikhonov regularisation

We can add a regularisation term to problem solved by CGLS; this gives us the minimisation problem in the following form, which is known as Tikhonov regularization:
$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}A u - b \end{Vmatrix}^2_2 + \alpha^2\|Lu\|^2_2$$

where,

- $A$ is the projection operator

- $b$ is the acquired data

- $u$ is the unknown image to be solved for

- $\alpha$ is the regularisation parameter

- $L$ is a regularisation operator


The first term measures the fidelity of the solution to the data. The second term meausures the fidelity to the prior knowledge we have imposed on the system, operator $L$. $\alpha$ controls the trade-off between these terms. $L$ is often chosen to be a smoothing operator like the identity matrix, or a gradient operator **constrained to the squared L2-norm**.

This can be re-written equivalently in the block matrix form:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\binom{A}{\alpha L} u - \binom{b}{0}\end{Vmatrix}^2_2$$

With the definitions:

- $\tilde{A} = \binom{A}{\alpha L}$

- $\tilde{b} = \binom{b}{0}$

this can now be recognised as a least squares problem:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\tilde{A} u - \tilde{b}\end{Vmatrix}^2_2$$

and being a least squares problem, it can be solved using CGLS with $\tilde{A}$ as operator and $\tilde{b}$ as data.

**ToDo: add citation**<br>



#### Introducing the block framework

We can construct $\tilde{A}$ and $\tilde{b}$ using the BlockFramework in the CIL.

$\tilde{A}$ is a (column) BlockOperator of size 2x1.

`BlockOperator(op0,op1,shape=(1,2))` results in a row block operator, which is not what we need here.


`BlockOperator(op0,op1)`
and
`BlockOperator(op0,op1,shape=(2,1))` result in a column block

$\tilde{b}$ is a BlockDataContainer

`BlockDataContainer(DataContainer0, DataContainer1)`

where we note that for BlockDataContainers we do not need to specify a shape.

<a id="section_CGLS_alpha"></a>
#### Reconstruct using CGLS and the identity operator

The simplest form of Tikhonov uses the identity matrix as the regularization operator. we use an identity matrix as our regularisation operator we are adding constraints on the size of the solution $u$

<span style="color:red;font-size:larger">**Exercise 2:**</span> Construct the BlockOperator $\tilde{A}$ using the idenity operator. Find the value of $\alpha$ that gives you the best solution. **ToDo high and low alpha**

In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Identity(ig)
alpha = 0.4

#operator_block = BlockOperator(  )
operator_block = BlockOperator( A, alpha * L)

In [ ]:
#define the data b
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate(0))

Run CGLS as before, but passing the BlockOperator and BlockDataContainer

In [ ]:
#setup CGLS with the Block Operator and Block DataContainer
x_init = ig.allocate(0)      
cgls_regularised = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_regularised.max_iteration = 1000

In [ ]:
#run the algorithm
cgls_regularised.run(100)

In [ ]:
#plot the results
CGLS_regularised_output = cgls_regularised.get_output()

plots = [CGLS_regularised_output, CGLS_regularised_output - model]
titles = ["CGLS reconstruction","Difference from ground truth" ]
plotter2D(plots,titles,fix_range=True)

In [ ]:
plt.plot(CGLS_simple_output.as_array()[512,:])
plt.plot(CGLS_regularised_output.as_array()[512,:])

### Using the BlockFramework to build a gradient operator

**ToDo: moved the bulk of the detailed explanation below, but I need something more here first. Think about explanation**

A gradient operator can be constructed using BlockOperators.

The direct gradient operator $\nabla$ acts on an image $u$ and returns a BlockDataContainer $\textbf{y}$

$$ \nabla(u) = 
\begin{bmatrix}
   \nabla_x\\
   \nabla_y\\
\end{bmatrix}
*u =
\begin{bmatrix}
    \nabla_xu\\
    \nabla_yu\\
\end{bmatrix}
=  
\begin{bmatrix}w_{x}\\w_{y}\end{bmatrix}= \textbf{w}$$

The adjoint gradient operator $\nabla^*$ acts on the BlockDataContainer $\textbf{y}$ and returns an image $\rho$

$$  \nabla^*(\textbf w) = 
\begin{bmatrix}
    \nabla^*_x &
    \nabla^*_y
\end{bmatrix}
*
\begin{bmatrix}
    w_{x}\\
    w_{y}\\
\end{bmatrix} 
=
\begin{bmatrix}
    \nabla^*_x w_x + \nabla^*_y w_y
\end{bmatrix} =  \rho$$

In [ ]:
#loading a test image
loader = TestData()
shapes = loader.load(TestData.SHAPES)
shapes_ig = shapes.geometry

#plot ths results
plotter2D(shapes, "shapes")

The finite difference operator can be called from the framework. This returns the difference between each pair of pixels along one direction.

We need to initialise it with the image geometry, the direction of the calculation and the boundary conditions to use.

`FiniteDiff(gm_domain, direction, bnd_cond='Neumann' or 'Periodic')`

In [ ]:
#define the operator FiniteDiff - needs to image geometry, the direction and the boundary conditions
fdx = FiniteDiff(shapes_ig, direction=1, bnd_cond='Neumann')

#run it over the input image
image_2D_dx = fdx.direct(shapes)

#plot ths results
plotter2D(image_2D_dx, "dx")

<span style="color:red;font-size:larger">**Exercise 3:**</span> Create a BlockOperator to containing the the finite difference operator in the $x$ and $y$ directions. Apply it to the test image.

In [ ]:
#define the x and y operators 
fdx = FiniteDiff(shapes_ig, direction=1, bnd_cond='Neumann')
fdy = FiniteDiff(shapes_ig, direction=0, bnd_cond='Neumann')

#fdx = 
#fdy = 

In [ ]:
#construct the BlockOperator
FD = BlockOperator(fdx, fdy)
#FD = #BlockOperator(fdx, fdy)

In [ ]:
#run it on the test image
fd_out = FD.direct(shapes)

In [ ]:
#plot the results
plots = [fd_out.get_item(0), fd_out.get_item(1)]
titles = ["dx","dy" ]
plotter2D(plots,titles)

A closer look at data types

In [ ]:
#input is ImageData
print("Input")
print("\ttype:\t", type(shapes))
print("\tshape:\t", shapes.shape)

In [ ]:
#output is BloackDataContainer
print("Output")
print("\ttype:\t", type(fd_out))
print("\tshape:\t", fd_out.shape)

print("\tDataContainer 0")
print("\t\ttype:\t", type(fd_out.get_item(0)))
print("\t\tshape:\t", fd_out.get_item(0).shape)

print("\tDataContainer 1")
print("\t\ttype:\t", type(fd_out.get_item(1)))
print("\t\tshape:\t", fd_out.get_item(1).shape)

In [ ]:
#split containers
dx = fd_out.get_item(0)
dy = fd_out.get_item(1)

#calculate the squared norm of the x and y gradients
dx2 = (dx**2).sum()
dy2 = (dy**2).sum()
sq_norm = dx2 + dy2
print(sq_norm)

In [ ]:
#do the same thing within the blockframework
sq_norm = fd_out.squared_norm()
print(sq_norm)

The BlockFramework provides basic algebra between BlockDataContainers, numpy arrays, lists of numbers,  DataContainers, subclasses and scalars providing the shape of the containers are compatible
- add
- subtract
- multiply
- divide
- power
- squared_norm

<span style="color:red;font-size:larger">**Exercise 4:**</span> Run the adjoint operator. What data type does it take in and write out?


In [ ]:
#run the adjoint method
adjoint_output = FD.adjoint(fd_out)

plotter2D(adjoint_output, "adjoint gradient")

### A deeper look at the BlockFramework
#### BlockDataContainer 

BlockDataContainer holds datacontainers as a column vector

$$\textbf{x} = \begin{bmatrix}x_{1}\\ x_{2}\end{bmatrix}$$

$$\textbf{y} = \begin{bmatrix}y_{1}\\ y_{2} \\ y_{3}\end{bmatrix}$$

#### BlockOperator: 

BlockOperator is a matrix of operators.

$$ K = \begin{bmatrix}
A_{1} & A_{2} \\
A_{3} & A_{4} \\
A_{5} & A_{6}
\end{bmatrix}_{(3,2)} *  \quad \underbrace{\begin{bmatrix}
x_{1} \\
x_{2} 
\end{bmatrix}_{(2,1)}}_{\textbf{x}} =  \begin{bmatrix}
A_{1}x_{1}  + A_{2}x_{2}\\
A_{3}x_{1}  + A_{4}x_{2}\\
A_{5}x_{1}  + A_{6}x_{2}\\
\end{bmatrix}_{(3,1)} =  \begin{bmatrix}
y_{1}\\
y_{2}\\
y_{3}
\end{bmatrix}_{(3,1)} = \textbf{y}$$

Column: Share the same domains $X_{1}, X_{2}$<br>
Rows: Share the same ranges $Y_{1}, Y_{2}, Y_{3}$

$$ K : (X_{1}\times X_{2}) \rightarrow (Y_{1}\times Y_{2} \times Y_{3})$$


$$ A_{1}, A_{3}, A_{5}: \text{share the same domain }  X_{1}$$
$$ A_{2}, A_{4}, A_{6}: \text{share the same domain }  X_{2}$$

$$A_{1}: X_{1} \rightarrow Y_{1}, \quad A_{3}: X_{1} \rightarrow Y_{2}, \quad  A_{5}: X_{1} \rightarrow Y_{3}$$
$$A_{2}: X_{2} \rightarrow Y_{1}, \quad A_{4}: X_{2} \rightarrow Y_{2}, \quad  A_{6}: X_{2} \rightarrow Y_{3}$$

### Reconstruct using Tikhonov CLGS with the gradient operator

#### Tikhonov regularisation

Now go back to our Tikonov reconstruction, this time define the gradient operator as the regulariser.

$${\mathrm{argmin}}\begin{Vmatrix}\binom{A}{\alpha \nabla} u - \binom{b}{0}\end{Vmatrix}^2_2$$

With the definitions:

- $\tilde{A} = \binom{A}{\alpha \nabla}$

- $\tilde{b} = \binom{b}{0}$

And solve using CGLS:
$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\tilde{A} u - \tilde{b}\end{Vmatrix}^2_2$$


We'll use the framework's `Gradient()` operator - this is an optimised form of FD over the space or space+channels dimensions.


**ToDo explain #L.range_geometry().allocate(0) explain!**


<span style="color:red;font-size:larger">**Exercise 5:**</span> Set up the BlockOperator $\tilde{A}$ and the BlockDataContainer $\tilde{b}$. Run Tikhonov reconstruction using gradient regularisation and find the value of $\alpha$ that gives you the best solution.

In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Gradient(ig)
alpha = 150

operator_block = BlockOperator( A, alpha * L, shape=(2,1))
#operator_block = BlockOperator( )

In [ ]:
#define the data b

#L.range_geometry().allocate(0) explain!

#data_block = BlockDataContainer()
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate(0))

In [ ]:
#setup CGLS with the block operator and block data
x_init = ig.allocate(0)      
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

In [ ]:
#run the algorithm
cgls_tikhonov.run(100, verbose = True)

In [ ]:
#plot the results
CGLS_tikhonov_output = cgls_tikhonov.get_output()

plots = [CGLS_tikhonov_output, CGLS_tikhonov_output - model]
titles = ["CGLS reconstruction","Difference from ground truth" ]
plotter2D(plots,titles,fix_range=True)

### Summary

#### Comparison of the outputs of each reconstruction

In [ ]:
#compare the outputs of unregularise and regularised CGLS
plots = [model, CGLS_simple_output, CGLS_regularised_output, CGLS_tikhonov_output]
titles = ["Ground truth", "CGLS simple","Tikhonov with Idenity regularisation","Tikhonov with gradient regularisation" ]
plotter2D(plots,titles,fix_range=False)

In [ ]:
plt.plot(model.as_array()[512,:])
plt.plot(CGLS_simple_output.as_array()[512,:])
plt.plot(CGLS_tikhonov_output.as_array()[512,:])

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularisation parameter and operator in regulariser.